# Import

In [1]:
import os
from time import sleep
from time import time as unixtime
from typing import Callable, List
import random
from math import ceil
import yaml
import string
import json
from urllib.parse import urlparse
from time import sleep
import re
from operator import itemgetter
import subprocess
from tempfile import NamedTemporaryFile

from dotenv import load_dotenv
from IPython.display import display, clear_output

import boto3
import kubernetes
from kubernetes.client.rest import ApiException

from pyhelm3 import Client as HelmClient


In [2]:
load_dotenv('/.env')

True

In [3]:
from helper import wait_until
from helper.ec2 import (get_vpcs_ids, get_internet_gateway_ids_attached_to_vpc, 
                        get_route_table_ids_for_vpc, route_to_gateway_exists, 
                        get_subnet_ids_in_vpc, get_security_group_ids)
from helper.k8s import (get_one_running_pod, get_jupyter_token_from_pod)

# Define

In [4]:
REGION = 'ca-central-1'
CLUSTER_NAME = 'kubyterlab-llm'
EBS_VOLUME_SIZE = 500 # GiB
TAGS = {'cluster': CLUSTER_NAME, 'purpose': 'llm'}  # Do not change the keys, they are hardcoded throughout.
CLUSTER_TAGS = {'cluster': CLUSTER_NAME}
VOLUME_FILTERS = [
    {'Name': f'tag:purpose', 
     'Values': ['kubyterlab-llm', 'llm']}]
K8S_VERSION = os.environ['K8S_VERSION']  # '1.30'
K8S_VERSION = '.'.join(K8S_VERSION.split('.')[:2]) if len(K8S_VERSION.split('.')) > 2 else K8S_VERSION
INSTANCE_TYPES = {'gpu': ['g4dn.2xlarge'], 'default': ['t3.medium']}
ALLOWED_PORTS = [80, 443, 22]

In [5]:
def check_all_vpcs_available(response: dict) -> True:
    if not 'Vpcs' in response:
        raise ValueError
    return all([vpc.get('State', '') == 'available' for vpc in response['Vpcs']])

In [6]:
def is_cluster_active(response: dict) -> bool:
    status = response['cluster']['status']
    clear_output(wait=True)
    display(status)
    return status == 'ACTIVE'


In [7]:
def is_node_group_active(response: dict) -> bool:
    status = response['nodegroup']['status']
    clear_output(wait=True)
    display(status)
    return status in ['ACTIVE', 'CREATE_FAILED']


In [8]:
def is_snapshot_completed(response: dict) -> bool:
    state = response['Snapshots'][0]['State']
    clear_output(wait=True)
    display(state)
    return state.lower() == 'completed'


In [9]:
def is_volume_available(response: dict) -> bool:
    state = response['Volumes'][0]['State']
    clear_output(wait=True)
    display(state)
    return state.lower() == 'available'


# Instantiate

In [10]:
session = boto3.Session(region_name=REGION)
eks_client = session.client('eks')
ec2_client = session.client('ec2')
iam_client = session.client('iam')

aws_account_id = boto3.client('sts').get_caller_identity().get('Account')

# Create or Restore Volume

In [11]:
try:
    response = ec2_client.describe_volumes(Filters=VOLUME_FILTERS)
    volumes = response.get('Volumes', [])
except RuntimeError:
    volumes = []
volume_ids = [volume['VolumeId'] for volume in volumes]
volume_ids

[]

In [12]:
assert len(volume_ids) <= 1  # TODO: Get the latest one if more than one.
if volume_ids:
    volume_id = volume_ids[0]
    availability_zone = volumes[0]['AvailabilityZone']
else:
    volume_id = None

In [13]:
if not volume_id:
    response = ec2_client.describe_snapshots(Filters=VOLUME_FILTERS)
    snapshots = response.get('Snapshots', [])
    if snapshots:
        sorted_snapshots = sorted(snapshots, key=itemgetter('StartTime'), reverse=True)
        snapshot_id = sorted_snapshots[0]['SnapshotId']
        print(snapshot_id)

snap-070f65816ea732b4e


In [14]:
if not volume_id:
    response = ec2_client.describe_availability_zones()
    availability_zones = response['AvailabilityZones']
    availability_zone = availability_zones[0]['ZoneName']
    # availability_zone = f'{REGION}a'
    if snapshots:
        # TODO: Change this to the latest snapshot!!
        response = ec2_client.create_volume(
            SnapshotId=snapshot_id,
            Size=EBS_VOLUME_SIZE,
            AvailabilityZone=availability_zone,
            VolumeType='gp3',
            TagSpecifications=[
                {
                    'ResourceType': 'volume',
                    'Tags': [{'Key': 'purpose', 'Value': 'llm'}]
                }
            ]
        )
    else:
        response = ec2_client.create_volume(
            Size=EBS_VOLUME_SIZE,
            AvailabilityZone=availability_zones[0]['ZoneName'],
            VolumeType='gp3',
            TagSpecifications=[
                {
                    'ResourceType': 'volume',
                    'Tags': [{'Key': 'purpose', 'Value': 'llm'}]
                }
            ]
        )
    wait_until(ec2_client.describe_volumes, {'VolumeIds': [response['VolumeId']]}, is_volume_available)
    volume_id = response['VolumeId']
volume_id, availability_zone

'available'

('vol-0b2f7361fa13c5275', 'ca-central-1a')

In [15]:
# TODO: Change the parts below to use Terraform

# Create VPC

In [16]:
vpc_ids = get_vpcs_ids(ec2_client, TAGS)
vpc_exists = len(vpc_ids) > 0

if len(vpc_ids) > 1:
    raise RuntimeError
elif vpc_exists:
    vpc_id = vpc_ids[0]

# Create VPC
if not vpc_exists:
    print('Creating VPC...')
    vpc_response = ec2_client.create_vpc(
        CidrBlock='10.0.0.0/16',
        TagSpecifications=[
            {
                'ResourceType': 'vpc',
                'Tags': [{'Key': tag, 'Value': TAGS[tag] }for tag in TAGS]
            }
        ]
    )
    vpc_id = vpc_response['Vpc']['VpcId']
wait_until(ec2_client.describe_vpcs, {'VpcIds': [vpc_id]}, check_all_vpcs_available)

# Create Internet Gateway
igw_ids = get_internet_gateway_ids_attached_to_vpc(ec2_client, vpc_id)
igw_exists = len(igw_ids) > 0

if len(igw_ids) > 1:
    raise RuntimeError
elif igw_exists:
    igw_id = igw_ids[0]


if not igw_exists:
    print('Creating Internet Gateway...')
    # Create an Internet Gateway
    igw_response = ec2_client.create_internet_gateway()
    igw_id = igw_response['InternetGateway']['InternetGatewayId']
    
    # Attach Internet Gateway to VPC
    ec2_client.attach_internet_gateway(
        InternetGatewayId=igw_id,
        VpcId=vpc_id
    )

# Create a route table
route_table_ids = get_route_table_ids_for_vpc(ec2_client, vpc_id)
route_table_exists = len(route_table_ids) > 0

if not route_table_exists:
    print('Creating a route table...')
    # Create a route table
    route_table_response = ec2_client.create_route_table(VpcId=vpc_id)
    route_table_id = route_table_response['RouteTable']['RouteTableId']

is_route_created = False
for route_table_id in route_table_ids:
    if route_to_gateway_exists(ec2_client, route_table_id, igw_id):
        is_route_created = True
        break

if not is_route_created:
    # Create a route to the Internet Gateway
    ec2_client.create_route(
        RouteTableId=route_table_id,
        DestinationCidrBlock='0.0.0.0/0',
        GatewayId=igw_id
    )

vpc_id, igw_id, route_table_id

Creating VPC...
Creating Internet Gateway...


('vpc-0d873bc388d2dfd91', 'igw-000c8b40a11ef13a8', 'rtb-0874add4d53244b3d')

# Create Subnets

In [17]:
subnet_ids = get_subnet_ids_in_vpc(ec2_client, vpc_id)
min_subnet_count = 2
current_subnet_count = len(subnet_ids)

# if current_subnet_count < min_subnet_count:
    # response = ec2_client.describe_availability_zones()
    
    # availability_zones = [az['ZoneName'] for az in response['AvailabilityZones']]
    # random.shuffle(availability_zones)
    # TODO: Following logic is still necessary for situations with larger subnets. Change to something reasonable.
    # availability_zones *= ceil(min_subnet_count / len(availability_zones))
    

while current_subnet_count < min_subnet_count:
    i = current_subnet_count + 1
    print(f'Creating subnet #{i}...')
    subnet_response = ec2_client.create_subnet(
        VpcId=vpc_id,
        CidrBlock=f'10.0.{i}.0/24',
        # AvailabilityZone=availability_zone,
        AvailabilityZone=availability_zones[current_subnet_count]['ZoneName'],
        TagSpecifications=[
            {
                'ResourceType': 'subnet',
                'Tags': [{'Key': tag, 'Value': TAGS[tag]} for tag in TAGS]
            }
        ]
    )
    subnet_id = subnet_response['Subnet']['SubnetId']
    subnet_ids.append(subnet_id)
    current_subnet_count = len(subnet_ids)

# check if any subnet is public, add the table to first subnet if not

Creating subnet #1...
Creating subnet #2...


In [18]:
subnets_response = ec2_client.describe_subnets(SubnetIds=subnet_ids)
assert len(subnets_response) == len(subnet_ids) == min_subnet_count
is_any_subnet_open_to_public = False
for subnet in subnets_response['Subnets']:
    if subnets_response.get('MapPublicIpOnLaunch', False):
        # I could also look at the route tables, and see if they are assigned to any subnet. ec2_client.describe_route_tables(RouteTableIds=[route_table_id]) RouteTables.SubnetId (optional parameter)
        public_subnet_id = subnet['SubnetId']
        is_any_subnet_open_to_public = True

if not is_any_subnet_open_to_public:
    for subnet in subnets_response['Subnets']:
        if subnet['CidrBlock'] == '10.0.1.0/24':
            public_subnet_id = subnet['SubnetId']

            # Associate the public subnet with the route table
            ec2_client.associate_route_table(
                RouteTableId=route_table_id,
                SubnetId=public_subnet_id
            )
            
            # Modify the public subnet to auto-assign public IPs
            ec2_client.modify_subnet_attribute(
                SubnetId=public_subnet_id,
                MapPublicIpOnLaunch={"Value": True}
            )
            break
            

# Create Security Group

In [19]:
security_group_ids = get_security_group_ids(ec2_client, TAGS)

security_group_exists = len(security_group_ids) > 0

if len(security_group_ids) > 1:
    raise RuntimeError
elif security_group_exists:
    security_group_id = security_group_ids[0]

# The security group gets torn down when deleted, so there is no need to check the rules and rewrite all of them.

if not security_group_exists:
    print('Creating Security Group...')
    response = ec2_client.create_security_group(
        GroupName=f'eks-cluster-sg-{CLUSTER_NAME}',
        TagSpecifications=[
            {
                'ResourceType': 'security-group', 
                'Tags': [{'Key': tag, 'Value': TAGS[tag]} for tag in TAGS]
            }
        ],
        Description=f'Security group for EKS cluster: {CLUSTER_NAME}',
        VpcId=vpc_id
    )
    security_group_id = response['GroupId']

    # TODO: Can I make the CidrIp more restrictive for the next deployment? Load Balancer needs to have a static IP, probably through the Kubernetes YAML?
    for port in ALLOWED_PORTS:
        ec2_client.authorize_security_group_ingress(
            GroupId=security_group_id,
            IpPermissions=[
                {
                    'IpProtocol': 'tcp',
                    'FromPort': port,
                    'ToPort': port,
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                }
            ]
        )
    
security_group_id

Creating Security Group...


'sg-0cc5d66a795596b94'

In [20]:
volume_id, vpc_id, igw_id, route_table_id, public_subnet_id, subnet_ids, security_group_id

('vol-0b2f7361fa13c5275',
 'vpc-0d873bc388d2dfd91',
 'igw-000c8b40a11ef13a8',
 'rtb-0874add4d53244b3d',
 'subnet-094faf3e919da233a',
 ['subnet-094faf3e919da233a', 'subnet-001821e9b0ed9d7e4'],
 'sg-0cc5d66a795596b94')

# Create Cluster

In [21]:
tear_down_and_rebuild = False
response = eks_client.list_clusters()
clusters = response['clusters']
if CLUSTER_NAME not in clusters:
    eks_client.create_cluster(name=CLUSTER_NAME, 
                              version=K8S_VERSION, 
                              roleArn=f'arn:aws:iam::{aws_account_id}:role/EKS_Cluster_Role', 
                              resourcesVpcConfig={'subnetIds': subnet_ids,
                                                  'securityGroupIds': [security_group_id],
                                                  'endpointPublicAccess': True,
                                                  'endpointPrivateAccess': False
                              },
                              tags=TAGS,
                             )
else:
    if tear_down_and_rebuild:
        pass
        # TODO: Create a new cluster with a temp name.
        # Wait until the cluster has finished forming
        # Delete the old cluster
        # Wait until deletion is complete
        # Rename the new cluster
        
wait_until(eks_client.describe_cluster, {'name': CLUSTER_NAME}, is_cluster_active, timeout=7 * 60)
response = eks_client.describe_cluster(name=CLUSTER_NAME)
assert response['cluster']['status'] == 'ACTIVE'

'ACTIVE'

# Create IAM Role for Node Groups

In [22]:
# Define the role name
role_name = 'EKS_Cluster_Role'  # WARNING: I get a weird error if I call this role anything else. The call looks for this particular name, and I do not know how to override it.

# Create the trust policy for the role
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "eks.amazonaws.com",
                    "ec2.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create the IAM role
try:
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Role for EKS Node Group'
    )
    node_role_arn = response['Role']['Arn']
    print(f"Created role: {node_role_arn}")
except iam_client.exceptions.EntityAlreadyExistsException:
    response = iam_client.get_role(RoleName=role_name)
    node_role_arn = response['Role']['Arn']
    print(f"Role {role_name} already exists. Arn: {node_role_arn}.")
    # TODO: Check if trust_policy is correct.

# Attach necessary policies
policies = [
    'AmazonEKSWorkerNodePolicy',
    'AmazonEC2ContainerRegistryReadOnly',
    'AmazonEKS_CNI_Policy',
    'AmazonEKSClusterPolicy',
    # 'AmazonSSMManagedInstanceCore',
]
# Policy AmazonSSMManagedInstanceCore is not necessary, I used it for debugging, to connect to the node and run commands. 

for policy in policies:
    try:
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=f'arn:aws:iam::aws:policy/{policy}'
        )
        print(f"Attached policy {policy} to role {role_name}.")
    except Exception as e:
        print(f"Error attaching policy {policy}: {e}")

Role EKS_Cluster_Role already exists. Arn: arn:aws:iam::275678099358:role/EKS_Cluster_Role.
Attached policy AmazonEKSWorkerNodePolicy to role EKS_Cluster_Role.
Attached policy AmazonEC2ContainerRegistryReadOnly to role EKS_Cluster_Role.
Attached policy AmazonEKS_CNI_Policy to role EKS_Cluster_Role.
Attached policy AmazonEKSClusterPolicy to role EKS_Cluster_Role.


# Add Node Groups

In [23]:
# TODO: This part may also need a wait.

In [24]:
node_groups = eks_client.list_nodegroups(clusterName=CLUSTER_NAME)
# TODO: Make this static and put to the top, with different `diskSize` and `scalingConfig`
node_group_name = 'gpu'
ami_type = 'AL2_x86_64_GPU'
if node_group_name in node_groups['nodegroups']:
    response = eks_client.describe_nodegroup(clusterName=CLUSTER_NAME, nodegroupName=node_group_name)
    status = response['nodegroup']['status']
    if status == 'CREATE_FAILED':
        print('Node group exists, but failed to create. Deleting...')
        failed_node_group_info = response

        eks_client.delete_nodegroup(clusterName=CLUSTER_NAME, nodegroupName=node_group_name)
    
        wait_until(eks_client.list_nodegroups, {'clusterName': CLUSTER_NAME}, lambda x: node_group_name not in x['nodegroups'])
        node_groups = eks_client.list_nodegroups(clusterName=CLUSTER_NAME)
    else:
        print('Node group exists.')



if not node_groups['nodegroups']:
    print('Creating node group...')
    response = eks_client.create_nodegroup(
        clusterName=CLUSTER_NAME,
        nodegroupName=node_group_name,
        scalingConfig={
            'desiredSize': 1,
            'minSize': 1,
            'maxSize': 1
        },
        diskSize=50,  # Size in GiB
        subnets=[public_subnet_id],
        nodeRole=node_role_arn,
        amiType=ami_type,
        instanceTypes=INSTANCE_TYPES[node_group_name],
        labels={
            'gpu-memory': 'true'
        },
        taints=[
            {
                'key': 'nvidia.com/gpu',
                'value': 'true',
                'effect': 'NO_SCHEDULE'
            }
        ]
    )
wait_until(eks_client.describe_nodegroup, {'clusterName': CLUSTER_NAME, 'nodegroupName': node_group_name}, is_node_group_active)

'CREATING'

False

In [25]:
node_groups = eks_client.list_nodegroups(clusterName=CLUSTER_NAME)
# TODO: Make this static and put to the top, with different `diskSize` and `scalingConfig`
node_group_name = 'default'
ami_type = 'AL2_x86_64'
if node_group_name in node_groups['nodegroups']:
    response = eks_client.describe_nodegroup(clusterName=CLUSTER_NAME, nodegroupName=node_group_name)
    status = response['nodegroup']['status']
    if status == 'CREATE_FAILED':
        print('Node group exists, but failed to create. Deleting...')
        failed_node_group_info = response

        eks_client.delete_nodegroup(clusterName=CLUSTER_NAME, nodegroupName=node_group_name)
    
        wait_until(eks_client.list_nodegroups, {'clusterName': CLUSTER_NAME}, lambda x: node_group_name not in x['nodegroups'])
        node_groups = eks_client.list_nodegroups(clusterName=CLUSTER_NAME)
    else:
        print('Node group exists.')


# TODO: See if lower disksize will work.
if node_group_name not in node_groups['nodegroups']:
    print('Creating node group...')
    response = eks_client.create_nodegroup(
        clusterName=CLUSTER_NAME,
        nodegroupName=node_group_name,
        scalingConfig={
            'desiredSize': 2,
            'minSize': 2,
            'maxSize': 2
        },
        diskSize=1000,  # Size in GiB
        subnets=[public_subnet_id],
        nodeRole=node_role_arn,
        amiType=ami_type,
        instanceTypes=INSTANCE_TYPES[node_group_name],
    )
wait_until(eks_client.describe_nodegroup, {'clusterName': CLUSTER_NAME, 'nodegroupName': node_group_name}, is_node_group_active)

'CREATING'

False

In [98]:
# This is for double-checking the type of instance.
asg_client = boto3.client("autoscaling")
node_group = eks_client.describe_nodegroup(clusterName=CLUSTER_NAME, nodegroupName='gpu')
asg_name = node_group["nodegroup"]["resources"]["autoScalingGroups"][0]["name"]
asg_details = asg_client.describe_auto_scaling_groups(AutoScalingGroupNames=[asg_name])
instance_ids = [instance["InstanceId"] for instance in asg_details["AutoScalingGroups"][0]["Instances"]]
for instance_id in instance_ids:
    instance_details = ec2_client.describe_instances(InstanceIds=[instance_id])
    instance_type = instance_details["Reservations"][0]["Instances"][0]["InstanceType"]
    print(f"Instance {instance_id} is of type {instance_type}")

Instance i-0da26cffc3155cdf8 is of type g4dn.xlarge


# Initialize Kubernetes

In [26]:
!aws eks update-kubeconfig --name $CLUSTER_NAME --region $REGION

Updated context arn:aws:eks:ca-central-1:275678099358:cluster/kubyterlab-llm in /root/.kube/config


# Initialize Helm

In [27]:
!helm repo add aws-ebs-csi-driver https://kubernetes-sigs.github.io/aws-ebs-csi-driver

"aws-ebs-csi-driver" already exists with the same configuration, skipping


In [28]:
!helm repo add eks https://aws.github.io/eks-charts

"eks" already exists with the same configuration, skipping


In [29]:
!helm repo add ingress-nginx https://kubernetes.github.io/ingress-nginx

"ingress-nginx" already exists with the same configuration, skipping


In [30]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "ingress-nginx" chart repository
...Successfully got an update from the "aws-ebs-csi-driver" chart repository
...Successfully got an update from the "eks" chart repository
Update Complete. ⎈Happy Helming!⎈


# Create a Role to Install the CSI Driver and Give Permissions Using the OIDC Provider

In [31]:
cluster_response = eks_client.describe_cluster(name=CLUSTER_NAME)
oidc_id = cluster_response['cluster']['identity']['oidc']['issuer'].split('/')[-1]
oidc_url = f'https://oidc.eks.{REGION}.amazonaws.com/id/{oidc_id}'
oidc_url

'https://oidc.eks.ca-central-1.amazonaws.com/id/AFE8CC0B699B011A4B5CD4E88D8834FC'

In [32]:
!eksctl utils associate-iam-oidc-provider --cluster $CLUSTER_NAME --approve

2025-03-04 02:50:35 [ℹ]  will create IAM Open ID Connect provider for cluster "kubyterlab-llm" in "ca-central-1"
2025-03-04 02:50:36 [✔]  created IAM Open ID Connect provider for cluster "kubyterlab-llm" in "ca-central-1"


In [33]:
amazon_eks_ebs_csi_driver_policy_name = 'AmazonEKS_EBS_CSI_Driver_Policy'
amazon_eks_ebs_csi_driver_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "ec2:CreateSnapshot",
        "ec2:AttachVolume",
        "ec2:DetachVolume",
        "ec2:ModifyVolume",
        "ec2:DescribeAvailabilityZones",
        "ec2:DescribeInstances",
        "ec2:DescribeSnapshots",
        "ec2:DescribeTags",
        "ec2:DescribeVolumes",
        "ec2:DescribeVolumesModifications"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:CreateTags"
      ],
      "Resource": [
        "arn:aws:ec2:*:*:volume/*",
        "arn:aws:ec2:*:*:snapshot/*"
      ],
      "Condition": {
        "StringEquals": {
          "ec2:CreateAction": [
            "CreateVolume",
            "CreateSnapshot"
          ]
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteTags"
      ],
      "Resource": [
        "arn:aws:ec2:*:*:volume/*",
        "arn:aws:ec2:*:*:snapshot/*"
      ]
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:CreateVolume"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "aws:RequestTag/ebs.csi.aws.com/cluster": "true"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:CreateVolume"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "aws:RequestTag/CSIVolumeName": "*"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteVolume"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "ec2:ResourceTag/CSIVolumeName": "*"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteVolume"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "ec2:ResourceTag/ebs.csi.aws.com/cluster": "true"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteSnapshot"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "ec2:ResourceTag/CSIVolumeSnapshotName": "*"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteSnapshot"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "ec2:ResourceTag/ebs.csi.aws.com/cluster": "true"
        }
      }
    }
  ]
}
try:
    response = iam_client.create_policy(
        PolicyName=amazon_eks_ebs_csi_driver_policy_name,
        PolicyDocument=json.dumps(amazon_eks_ebs_csi_driver_policy)
    )
    amazon_eks_ebs_csi_driver_policy_arn = response['Policy']['Arn']
except iam_client.exceptions.EntityAlreadyExistsException:
    amazon_eks_ebs_csi_driver_policy_arn = f'arn:aws:iam::{aws_account_id}:policy/{amazon_eks_ebs_csi_driver_policy_name}'
    response = iam_client.get_policy(PolicyArn=amazon_eks_ebs_csi_driver_policy_arn)

    # TODO: If policy body is not the same, update. Code to update follows.
    # iam_client.create_policy_version(
    #     PolicyArn=policy_arn,
    #     PolicyDocument=policy_document_json,
    #     SetAsDefault=True
    # )

amazon_eks_ebs_csi_driver_policy_arn

'arn:aws:iam::275678099358:policy/AmazonEKS_EBS_CSI_Driver_Policy'

In [34]:
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Federated": f"arn:aws:iam::{aws_account_id}:oidc-provider/oidc.eks.{REGION}.amazonaws.com/id/{oidc_id}"
            },
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {
                "StringEquals": {
                    f"oidc.eks.{REGION}.amazonaws.com/id/{oidc_id}:aud": "sts.amazonaws.com",
                    f"oidc.eks.{REGION}.amazonaws.com/id/{oidc_id}:sub": "system:serviceaccount:kube-system:ebs-csi-controller-sa"
                }
            }
        }
    ]
}

try:
    create_role_response = iam_client.create_role(
        RoleName='AmazonEKS_EBS_CSI_DriverRole',
        
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Role for Amazon EKS EBS CSI Driver'
    )
    print(f"Role created successfully: {create_role_response['Role']['Arn']}")
except iam_client.exceptions.EntityAlreadyExistsException:
    print("Role already exists. Updating the trust policy.")
    iam_client.update_assume_role_policy(
        RoleName='AmazonEKS_EBS_CSI_DriverRole',
        PolicyDocument=json.dumps(trust_policy)
    )

attach_policy_response = iam_client.attach_role_policy(
    RoleName='AmazonEKS_EBS_CSI_DriverRole',
    PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonEBSCSIDriverPolicy'
)

attach_policy_response = iam_client.attach_role_policy(
    RoleName='AmazonEKS_EBS_CSI_DriverRole',
    PolicyArn=amazon_eks_ebs_csi_driver_policy_arn
)

Role already exists. Updating the trust policy.


# Install CSI Driver

In [35]:
!kubectl create sa ebs-csi-controller-sa -n kube-system

serviceaccount/ebs-csi-controller-sa created


In [36]:
!kubectl annotate sa ebs-csi-controller-sa -n kube-system eks.amazonaws.com/role-arn=$ebs_csi_driver_role_arn


serviceaccount/ebs-csi-controller-sa annotated


In [37]:
!kubectl annotate sa ebs-csi-controller-sa -n kube-system meta.helm.sh/release-namespace=kube-system

serviceaccount/ebs-csi-controller-sa annotated


In [38]:
!kubectl annotate sa ebs-csi-controller-sa -n kube-system meta.helm.sh/release-name=aws-ebs-csi-driver

serviceaccount/ebs-csi-controller-sa annotated


In [39]:
!kubectl label sa ebs-csi-controller-sa -n kube-system app.kubernetes.io/managed-by=Helm

serviceaccount/ebs-csi-controller-sa not labeled


In [40]:
!helm upgrade --install aws-ebs-csi-driver \
    --namespace kube-system \
    aws-ebs-csi-driver/aws-ebs-csi-driver

Release "aws-ebs-csi-driver" has been upgraded. Happy Helming!
NAME: aws-ebs-csi-driver
LAST DEPLOYED: Tue Mar  4 02:50:41 2025
NAMESPACE: kube-system
STATUS: deployed
REVISION: 2
NOTES:
To verify that aws-ebs-csi-driver has started, run:

    kubectl get pod -n kube-system -l "app.kubernetes.io/name=aws-ebs-csi-driver,app.kubernetes.io/instance=aws-ebs-csi-driver"

[Deprecation announcement] AWS Snow Family device support for the EBS CSI Driver

Support for the EBS CSI Driver on [AWS Snow Family devices](https://aws.amazon.com/snowball/) is deprecated, effective immediately. No further Snow-specific bugfixes or feature requests will be merged. The existing functionality for Snow devices will be removed the 1.43 release of the EBS CSI Driver. This announcement does not affect the support of the EBS CSI Driver on other platforms, such as [Amazon EC2](https://aws.amazon.com/ec2/) or EC2 on [AWS Outposts](https://aws.amazon.com/outposts/). For any questions related to this announcement, p

### == End of Code Specific to AWS ==

# Install Nvidia Device Plugin

In [41]:
!kubectl apply -f https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/refs/heads/main/deployments/static/nvidia-device-plugin.yml

daemonset.apps/nvidia-device-plugin-daemonset unchanged


# Apply Helm Charts

In [42]:
volume_id

'vol-0b2f7361fa13c5275'

In [43]:
!helm install storage /helm/storage/ --set volumeID=$volume_id

NAME: storage
LAST DEPLOYED: Tue Mar  4 02:50:45 2025
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [44]:
!helm install ingress-nginx ingress-nginx/ingress-nginx --namespace ingress-nginx --create-namespace

NAME: ingress-nginx
LAST DEPLOYED: Tue Mar  4 02:50:46 2025
NAMESPACE: ingress-nginx
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
The ingress-nginx controller has been installed.
It may take a few minutes for the load balancer IP to be available.
You can watch the status by running 'kubectl get service --namespace ingress-nginx ingress-nginx-controller --output wide --watch'

An example Ingress that makes use of the controller:
  apiVersion: networking.k8s.io/v1
  kind: Ingress
  metadata:
    name: example
    namespace: foo
  spec:
    ingressClassName: nginx
    rules:
      - host: www.example.com
        http:
          paths:
            - pathType: Prefix
              backend:
                service:
                  name: exampleService
                  port:
                    number: 80
              path: /
    # This section is only required if TLS is to be enabled for the Ingress
    tls:
      - hosts:
        - www.example.com
        secretName: example-tls

In [45]:
# !helm upgrade --install lb /helm/lb-ingress/


In [46]:
# !helm delete lb

In [47]:
# TODO: Need to wait before applying the ingress

In [48]:
!helm upgrade --install fastapi /helm/eberron-agent-server/ --set awsAccountId=$aws_account_id --set region=$REGION


Release "fastapi" does not exist. Installing it now.
NAME: fastapi
LAST DEPLOYED: Tue Mar  4 02:51:15 2025
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [49]:
!helm upgrade --install frontend /helm/eberron-agent-frontend/ --set awsAccountId=$aws_account_id --set region=$REGION


Release "frontend" does not exist. Installing it now.
NAME: frontend
LAST DEPLOYED: Tue Mar  4 02:51:17 2025
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [73]:
!helm upgrade --install ingress /helm/eberron-agent-ingress/


Release "ingress" has been upgraded. Happy Helming!
NAME: ingress
LAST DEPLOYED: Tue Mar  4 04:06:14 2025
NAMESPACE: default
STATUS: deployed
REVISION: 10
TEST SUITE: None


# Get URL

In [64]:
# TODO: Update get_one_running_pod to use staring with names.
wait_until(get_one_running_pod, {'prefix': 'fastapi-'}, lambda x: x is not None, timeout=8 * 60)
pod_name = get_one_running_pod(prefix='fastapi-')
assert pod_name is not None
pod_name

'fastapi-677965765c-6cbjw'

In [65]:
# output = !kubectl get service lb --no-headers
# fields = re.split(r'\s+', output[0])
# external_url = fields[3]
# port = fields[4].split('/')[0].split(':')[0]
# external_url, port

In [66]:

output = !kubectl get service ingress-nginx-controller --namespace ingress-nginx --no-headers
fields = re.split(r'\s+', output[0])
external_url = fields[3]
port = fields[4].split('/')[0].split(':')[0]
external_url, port

('ab2781576bfaa46ac9445ed997849abb-1499725304.ca-central-1.elb.amazonaws.com',
 '80')

In [67]:
url = f'http://{external_url}:{port}'
url

'http://ab2781576bfaa46ac9445ed997849abb-1499725304.ca-central-1.elb.amazonaws.com:80'

# Test

In [74]:
# Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
# True
# 1
!kubectl exec $pod_name bash -- python3 -c 'import torch; print(torch.cuda.is_available()); print(torch.cuda.device_count())'

True
1


In [75]:
!kubectl exec $pod_name bash -- bash -c 'nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv'

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 909 MiB, 14187 MiB


In [77]:
import requests

from IPython.display import clear_output, display, HTML, Markdown

response = requests.post(f"{url}/respond", 
                         headers={"Content-Type": "application/json", 
                                  "Accept": "text/event-stream"}, 
                         json={"content": "What are the languages in Eberron?"}, 
                         stream=True)
md = ""
for chunk in response:
    md += chunk.decode("utf-8")
    clear_output()
    display(Markdown(md.replace('\\n', '\n')))

In Eberron, there are various languages that reflect culture, geography, and the planes of existence. Some of the common languages include:

1. Common: The language of the Five Nations and the language of trade in Khorvaire, known by most of its people.
2. Abyssal: The common tongue of all fiends, sometimes called “Khyber’s Speech.”
3. Celestial: The tongue of Siberys, used by archons of Shavarath'.
4. Draconic: Used by dragons, kobolds, troglodytes, lizardfolk, and other draconic creatures.
5. Elven: Used by elves and drow.
6. Dwarven: Used by dwarves.
7. Goblin: The trade language of the goblin empire of Dhakaan and survives as the primary language in Darguun, Droaam, and the Shadow Marches.
8. Halfling: The language of halflings.
9. Ignan: The language of fire-based creatures.
10. Infernal: The language of devils of Shavarath'.
11. Kythric: The language of Slaadi, chaotic outsiders.
12. Mabran: The language of Nightshades, shadows, and Draconic creatures of Mabar.
13. Ore: The language of Orcs.
14. Quori: The language of Quori, the Inspired, and kalashtar.
15. Risian: The language of ice-based creatures.
16. Sylvan: The language of dryads, eladrins, and creatures of Thelanis.
17. Terran: The language of Xorns and other earth-based creatures.
18. Undercommon: The language of chokers and underground Daelkyr denizens.

These languages have their own unique alphabets and scripts, and some languages are more commonly spoken in certain regions or among certain races. The historical development of languages and cultures also plays a role in the scripts used to write various languages. For example, the Orc language is written using the Goblin script, because the orcs of Khorvaire learned writing from the goblins.

# == End of Procedure ==

## == Do Not Continue: rest of the code is only for reference for troubleshooting ==

# Troubleshoot

In [ ]:
# curl -X POST -H "Content-Type: application/json" -H "Accept: text/event-stream" -d '{"key":"value"}' http://your-api-endpoint

In [ ]:
# Note that for the following to work, you need to attach the policy AmazonSSMManagedInstanceCore, in the relevant cell above, during the deployment.

In [51]:
output = !aws ssm send-command \
    --document-name "AWS-RunShellScript" \
    --targets "Key=instanceIds,Values=i-02f3c1f8a118eb352" \
    --parameters 'commands=["nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv"]' \
    --region $REGION
command_id = json.loads(''.join(output))['Command']['CommandId']
command_id


In [ ]:
output = !aws ssm list-command-invocations \
    --command-id $command_id \
    --details
output = json.loads(''.join(output))['CommandInvocations'][0]['CommandPlugins'][0]['Output']
for line in output.split('\n'):
    print(line)

In [58]:
!kubectl get events --sort-by='.lastTimestamp'


LAST SEEN   TYPE      REASON                    OBJECT                                             MESSAGE
38m         Normal    Starting                  node/ip-10-0-1-19.ca-central-1.compute.internal    
39m         Normal    Starting                  node/ip-10-0-1-118.ca-central-1.compute.internal   
38m         Normal    Starting                  node/ip-10-0-1-35.ca-central-1.compute.internal    
39m         Normal    NodeAllocatableEnforced   node/ip-10-0-1-118.ca-central-1.compute.internal   Updated Node Allocatable limit across pods
39m         Normal    NodeHasSufficientPID      node/ip-10-0-1-118.ca-central-1.compute.internal   Node ip-10-0-1-118.ca-central-1.compute.internal status is now: NodeHasSufficientPID
39m         Normal    NodeHasNoDiskPressure     node/ip-10-0-1-118.ca-central-1.compute.internal   Node ip-10-0-1-118.ca-central-1.compute.internal status is now: NodeHasNoDiskPressure
39m         Normal    NodeHasSufficientMemory   node/ip-10-0-1-118.ca-central-1.co

In [94]:
instance_id

NameError: name 'instance_id' is not defined

In [59]:
!kubectl get pod -n kube-system -l "app.kubernetes.io/name=aws-ebs-csi-driver,app.kubernetes.io/instance=aws-ebs-csi-driver"

NAME                                  READY   STATUS    RESTARTS   AGE
ebs-csi-controller-6cd4bccd45-272bl   5/5     Running   0          38m
ebs-csi-controller-6cd4bccd45-g4vws   5/5     Running   0          38m
ebs-csi-node-lnggw                    3/3     Running   0          38m
ebs-csi-node-psgw7                    3/3     Running   0          38m
ebs-csi-node-qtmfb                    3/3     Running   0          38m


In [60]:
# NAME                                          GPU
# ip-10-0-1-129.ca-central-1.compute.internal   <none>
# ip-10-0-1-223.ca-central-1.compute.internal   <none>
# ip-10-0-1-8.ca-central-1.compute.internal     1
!kubectl get nodes "-o=custom-columns=NAME:.metadata.name,GPU:.status.allocatable.nvidia\.com/gpu"


NAME                                          GPU
ip-10-0-1-118.ca-central-1.compute.internal   1
ip-10-0-1-19.ca-central-1.compute.internal    <none>
ip-10-0-1-35.ca-central-1.compute.internal    <none>


In [61]:
# NAME                                          TAINTS
# ip-10-0-1-107.ca-central-1.compute.internal   <none>
# ip-10-0-1-175.ca-central-1.compute.internal   [map[effect:NoSchedule key:nvidia.com/gpu value:true]]
# ip-10-0-1-90.ca-central-1.compute.internal    <none>
!kubectl get nodes -o custom-columns=NAME:.metadata.name,TAINTS:.spec.taints

NAME                                          TAINTS
ip-10-0-1-118.ca-central-1.compute.internal   [map[effect:NoSchedule key:nvidia.com/gpu value:true]]
ip-10-0-1-19.ca-central-1.compute.internal    <none>
ip-10-0-1-35.ca-central-1.compute.internal    <none>


In [62]:
# NAME                     TYPE           CLUSTER-IP     EXTERNAL-IP                                                                  PORT(S)          AGE
# kubernetes               ClusterIP      172.20.0.1     <none>                                                                       443/TCP          37m
# kubyterlab-llm-service   LoadBalancer   172.20.129.8   a4740e3e56bfe40ac81121bd46071903-1377611187.ca-central-1.elb.amazonaws.com   8888:31434/TCP   13s
!kubectl get service

NAME         TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
fastapi      ClusterIP   172.20.116.75   <none>        80/TCP    38m
frontend     ClusterIP   172.20.41.240   <none>        80/TCP    38m
kubernetes   ClusterIP   172.20.0.1      <none>        443/TCP   42m


In [63]:
!kubectl get pod

NAME                        READY   STATUS    RESTARTS   AGE
fastapi-677965765c-6cbjw    1/1     Running   0          38m
frontend-7dc87ddddd-6x8q8   1/1     Running   0          38m


In [62]:
# NAME               READY   UP-TO-DATE   AVAILABLE   AGE
# kubyterlab-llm-pod   1/1     1            1           31m
!kubectl get deployments

NAME       READY   UP-TO-DATE   AVAILABLE   AGE
fastapi    1/1     1            1           104m
frontend   1/1     1            1           104m


In [71]:
!kubectl get ingress

No resources found in default namespace.


In [72]:
!kubectl get namespace

NAME              STATUS   AGE
default           Active   78m
ingress-nginx     Active   75m
kube-node-lease   Active   78m
kube-public       Active   78m
kube-system       Active   78m


In [68]:
!kubectl get ingress --show-labels

NAME      CLASS   HOSTS       ADDRESS                                                                      PORTS   AGE     LABELS
ingress   nginx   localhost   a16eb9ebc41444b9c8b8a9f27f4f47c7-1630223309.ca-central-1.elb.amazonaws.com   80      2m42s   app.kubernetes.io/managed-by=Helm


In [69]:
!kubectl get svc -n ingress-nginx


NAME                                 TYPE           CLUSTER-IP      EXTERNAL-IP                                                                  PORT(S)                      AGE
ingress-nginx-controller             LoadBalancer   172.20.74.194   a16eb9ebc41444b9c8b8a9f27f4f47c7-1630223309.ca-central-1.elb.amazonaws.com   80:31683/TCP,443:30812/TCP   107m
ingress-nginx-controller-admission   ClusterIP      172.20.15.158   <none>                                                                       443/TCP                      107m


In [91]:
!kubectl get nodes

NAME                                          STATUS   ROLES    AGE   VERSION
ip-10-0-1-118.ca-central-1.compute.internal   Ready    <none>   84m   v1.31.5-eks-5d632ec
ip-10-0-1-19.ca-central-1.compute.internal    Ready    <none>   83m   v1.31.5-eks-5d632ec
ip-10-0-1-35.ca-central-1.compute.internal    Ready    <none>   83m   v1.31.5-eks-5d632ec


In [70]:
!kubectl logs -l app.kubernetes.io/name=ingress-nginx -n ingress-nginx


I0226 19:21:54.285668       7 controller.go:196] "Configuration changes detected, backend reload required"
I0226 19:21:54.385355       7 controller.go:216] "Backend successfully reloaded"
I0226 19:21:54.385954       7 event.go:377] Event(v1.ObjectReference{Kind:"Pod", Namespace:"ingress-nginx", Name:"ingress-nginx-controller-7657f6db5f-kxrd8", UID:"8cf97803-4eea-4e08-a9d8-b7873de2222b", APIVersion:"v1", ResourceVersion:"3848", FieldPath:""}): type: 'Normal' reason: 'RELOAD' NGINX reload triggered due to a change in configuration
2025/02/26 19:22:01 [error] 99#99: *42838 connect() failed (111: Connection refused) while connecting to upstream, client: 10.0.1.216, server: _, request: "GET / HTTP/1.1", upstream: "http://10.0.1.4:4200/", host: "a16eb9ebc41444b9c8b8a9f27f4f47c7-1630223309.ca-central-1.elb.amazonaws.com", referrer: "http://localhost:8890/"
2025/02/26 19:22:01 [error] 99#99: *42838 connect() failed (111: Connection refused) while connecting to upstream, client: 10.0.1.216, ser

In [119]:
!helm list -A


NAME              	NAMESPACE    	REVISION	UPDATED                                	STATUS  	CHART                    	APP VERSION
aws-ebs-csi-driver	kube-system  	1       	2025-02-26 17:25:46.294689009 +0000 UTC	deployed	aws-ebs-csi-driver-2.40.1	1.40.0     
fastapi           	default      	4       	2025-02-26 19:37:58.362187382 +0000 UTC	deployed	fastapi-0.1.0            	25.02      
frontend          	default      	5       	2025-02-26 19:38:00.37656958 +0000 UTC 	deployed	frontend-0.1.0           	25.02      
ingress           	default      	8       	2025-02-26 19:37:56.093484512 +0000 UTC	deployed	ingress-0.1.0            	25.02      
ingress-nginx     	ingress-nginx	1       	2025-02-26 17:36:55.117771706 +0000 UTC	deployed	ingress-nginx-4.12.0     	1.12.0     
storage           	default      	1       	2025-02-26 17:36:53.654556216 +0000 UTC	deployed	storage-0.1.0            	           


In [72]:
!helm get values ingress-nginx -n ingress-nginx

USER-SUPPLIED VALUES:
null


In [73]:
!kubectl get ingressclass

NAME    CONTROLLER             PARAMETERS   AGE
nginx   k8s.io/ingress-nginx   <none>       108m


In [92]:
!kubectl describe ingress ingress

Name:             ingress
Labels:           app.kubernetes.io/managed-by=Helm
Namespace:        default
Address:          ab2781576bfaa46ac9445ed997849abb-1499725304.ca-central-1.elb.amazonaws.com
Ingress Class:    nginx
Default backend:  <default>
Rules:
  Host        Path  Backends
  ----        ----  --------
  *           
              /respond   fastapi:80 (10.0.1.216:8000)
              /          frontend:80 (10.0.1.159:8501)
Annotations:  meta.helm.sh/release-name: ingress
              meta.helm.sh/release-namespace: default
              nginx.ingress.kubernetes.io/cors-allow-methods: POST,GET,PUT,DELETE,OPTIONS
              nginx.ingress.kubernetes.io/enable-methods: POST,GET,PUT,DELETE,OPTIONS
              nginx.ingress.kubernetes.io/proxy-read-timeout: 3600
              nginx.ingress.kubernetes.io/proxy-send-timeout: 3600
Events:
  Type    Reason  Age                From                      Message
  ----    ------  ----               ----                      -------

In [80]:
wait_until(get_one_running_pod, {'prefix': 'frontend-'}, lambda x: x is                                       not None, timeout=8 * 60)
frontend_pod_name = get_one_running_pod(prefix='frontend-')
assert frontend_pod_name is not None
frontend_pod_name

'frontend-7dc87ddddd-6x8q8'

In [81]:
!kubectl logs $frontend_pod_name




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.0.1.139:8501
  External URL: http://15.223.2.202:8501



In [82]:
!kubectl delete pod $frontend_pod_name

pod "frontend-7dc87ddddd-6x8q8" deleted


In [93]:
!kubectl get nodes

NAME                                          STATUS   ROLES    AGE   VERSION
ip-10-0-1-118.ca-central-1.compute.internal   Ready    <none>   93m   v1.31.5-eks-5d632ec
ip-10-0-1-19.ca-central-1.compute.internal    Ready    <none>   92m   v1.31.5-eks-5d632ec
ip-10-0-1-35.ca-central-1.compute.internal    Ready    <none>   92m   v1.31.5-eks-5d632ec


In [83]:
output = !kubectl get pods -n ingress-nginx --no-headers
fields = re.split(r'\s+', output[0])
ingress_controller_pod = fields[0]
ingress_controller_pod

'ingress-nginx-controller-7657f6db5f-sc4l6'

In [84]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- curl -v frontend.default.svc.cluster.local:80

* Host frontend.default.svc.cluster.local:80 was resolved.
* IPv6: (none)
* IPv4: 172.20.41.240
*   Trying 172.20.41.240:80...
* Connected to frontend.default.svc.cluster.local (172.20.41.240) port 80
* using HTTP/1.x
> GET / HTTP/1.1
> Host: frontend.default.svc.cluster.local
> User-Agent: curl/8.11.1
> Accept: */*
> 
* Request completely sent off
< HTTP/1.1 200 OK
< Server: TornadoServer/6.4.2
< Content-Type: text/html
< Date: Tue, 04 Mar 2025 04:12:02 GMT
< Accept-Ranges: bytes
< Etag: "dbab17a21eaa5481e5ea74ed9b2a58f9cddc1fa1c62651ba7aaaf6016c1781d9d3404eda0c9aa965c79e88d417929d1dfdda09325d0e845b3b42b7b0b6a642cd"
< Last-Modified: Thu, 27 Feb 2025 03:24:13 GMT
< Cache-Control: no-cache
< Content-Length: 1837
< Vary: Accept-Encoding
< 
<!--
 Copyright (c) Streamlit Inc. (2018-2022) Snowflake Inc. (2022-2025)

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

    

In [85]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- curl -v frontend.default.svc.cluster.local:80

* Host frontend.default.svc.cluster.local:80 was resolved.
* IPv6: (none)
* IPv4: 172.20.41.240
*   Trying 172.20.41.240:80...
* Connected to frontend.default.svc.cluster.local (172.20.41.240) port 80
* using HTTP/1.x
> GET / HTTP/1.1
> Host: frontend.default.svc.cluster.local
> User-Agent: curl/8.11.1
> Accept: */*
> 
* Request completely sent off
< HTTP/1.1 200 OK
< Server: TornadoServer/6.4.2
< Content-Type: text/html
< Date: Tue, 04 Mar 2025 04:12:03 GMT
< Accept-Ranges: bytes
< Etag: "dbab17a21eaa5481e5ea74ed9b2a58f9cddc1fa1c62651ba7aaaf6016c1781d9d3404eda0c9aa965c79e88d417929d1dfdda09325d0e845b3b42b7b0b6a642cd"
< Last-Modified: Thu, 27 Feb 2025 03:24:13 GMT
< Cache-Control: no-cache
< Content-Length: 1837
< Vary: Accept-Encoding
< 
<!--
 Copyright (c) Streamlit Inc. (2018-2022) Snowflake Inc. (2022-2025)

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

    

In [86]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- curl -v fastapi.default.svc.cluster.local:80

* Host fastapi.default.svc.cluster.local:80 was resolved.
* IPv6: (none)
* IPv4: 172.20.116.75
*   Trying 172.20.116.75:80...
* Connected to fastapi.default.svc.cluster.local (172.20.116.75) port 80
* using HTTP/1.x
> GET / HTTP/1.1
> Host: fastapi.default.svc.cluster.local
> User-Agent: curl/8.11.1
> Accept: */*
> 
* Request completely sent off
< HTTP/1.1 200 OK
< date: Tue, 04 Mar 2025 04:12:03 GMT
< server: uvicorn
< content-length: 27
< content-type: application/json
< 
* Connection #0 to host fastapi.default.svc.cluster.local left intact
{"message":"Hello, world!"}

In [87]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- curl -v -X POST fastapi.default.svc.cluster.local:8000/respond --no-buffer

* Host fastapi.default.svc.cluster.local:8000 was resolved.
* IPv6: (none)
* IPv4: 172.20.116.75
*   Trying 172.20.116.75:8000...
* connect to 172.20.116.75 port 8000 from 10.0.1.140 port 49998 failed: Operation timed out
* Failed to connect to fastapi.default.svc.cluster.local port 8000 after 130824 ms: Could not connect to server
* closing connection #0
curl: (28) Failed to connect to fastapi.default.svc.cluster.local port 8000 after 130824 ms: Could not connect to server
command terminated with exit code 28


In [88]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- curl -v -X POST fastapi.default.svc.cluster.local:8000/respond --no-buffer -H "Content-Type: application/json" -d '{"content": "What are the languages in Eberron?"}'

error: pod, type/name or --filename must be specified


In [89]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- curl -v 10.0.1.49:8000

*   Trying 10.0.1.49:8000...
* connect to 10.0.1.49 port 8000 from 10.0.1.140 port 58704 failed: Host is unreachable
* Failed to connect to 10.0.1.49 port 8000 after 3060 ms: Could not connect to server
* closing connection #0
curl: (7) Failed to connect to 10.0.1.49 port 8000 after 3060 ms: Could not connect to server
command terminated with exit code 7


In [90]:
# !kubectl exec $ingress_controller_pod -n ingress-nginx -- cat /etc/nginx/nginx.conf

In [192]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- curl -v -x POST 10.0.1.49:8000/respond --no-buffer -H "Content-Type: application/json" -d '{"content": "What are the languages in Eberron?"}'

* Could not resolve proxy: POST
* shutting down connection #0
curl: (5) Could not resolve proxy: POST
command terminated with exit code 5


In [79]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- nslookup fastapi.default.svc.cluster.local

error: pod, type/name or --filename must be specified


In [178]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- nslookup fastapi.default.svc.cluster.local

Server:		172.20.0.10
Address:	172.20.0.10:53


Name:	fastapi.default.svc.cluster.local
Address: 172.20.157.106



In [179]:
!kubectl get svc fastapi -n default

NAME      TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
fastapi   ClusterIP   172.20.157.106   <none>        8000/TCP   82m


In [181]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- curl -v 172.20.157.106:8000

*   Trying 172.20.157.106:8000...
* Connected to 172.20.157.106 (172.20.157.106) port 8000
* using HTTP/1.x
> GET / HTTP/1.1
> Host: 172.20.157.106:8000
> User-Agent: curl/8.11.1
> Accept: */*
> 
* Request completely sent off
< HTTP/1.1 200 OK
< date: Sat, 22 Feb 2025 21:16:29 GMT
< server: uvicorn
< content-length: 27
< content-type: application/json
< 
* Connection #0 to host 172.20.157.106 left intact
{"message":"Hello, world!"}

In [193]:
!kubectl exec -it $pod_name -n default -- curl -v fastapi:8000

*   Trying 172.20.157.106:8000...
* Connected to fastapi (172.20.157.106) port 8000 (#0)
> GET / HTTP/1.1
> Host: fastapi:8000
> User-Agent: curl/7.81.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< date: Sat, 22 Feb 2025 21:25:46 GMT
< server: uvicorn
< content-length: 27
< content-type: application/json
< 
* Connection #0 to host fastapi left intact
{"message":"Hello, world!"}

In [196]:
!kubectl exec -it $pod_name -n default -- curl -v -X POST fastapi:8000/respond --no-buffer 

*   Trying 172.20.157.106:8000...
* Connected to fastapi (172.20.157.106) port 8000 (#0)
> POST /respond HTTP/1.1
> Host: fastapi:8000
> User-Agent: curl/7.81.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 422 Unprocessable Entity
< date: Sat, 22 Feb 2025 21:26:46 GMT
< server: uvicorn
< content-length: 82
< content-type: application/json
< 
{"detail":[{"type":"missing","loc":["body"],"msg":"Field required","input":null}]}* Connection #0 to host fastapi left intact


In [ ]:
!kubectl exec -it $pod_name -n default -- curl -v -X POST fastapi:8000/respond --no-buffer -H "Content-Type: application/json" -d '{"content": "What are the languages in Eberron?"}'

In [183]:
!kubectl exec -it $ingress_controller_pod -n ingress-nginx -- curl -v fastapi.default.svc.cluster.local:8000

* Host fastapi.default.svc.cluster.local:8000 was resolved.
* IPv6: (none)
* IPv4: 172.20.157.106
*   Trying 172.20.157.106:8000...
* Connected to fastapi.default.svc.cluster.local (172.20.157.106) port 8000
* using HTTP/1.x
> GET / HTTP/1.1
> Host: fastapi.default.svc.cluster.local:8000
> User-Agent: curl/8.11.1
> Accept: */*
> 
* Request completely sent off
< HTTP/1.1 200 OK
< date: Sat, 22 Feb 2025 21:19:01 GMT
< server: uvicorn
< content-length: 27
< content-type: application/json
< 
* Connection #0 to host fastapi.default.svc.cluster.local left intact
{"message":"Hello, world!"}

In [162]:
!kubectl get endpoints fastapi -n default

NAME      ENDPOINTS        AGE
fastapi   10.0.1.49:8000   77m


In [ ]:
# !kubectl describe deployment kubyterlab-llm-pod

In [71]:
# !kubectl describe pod frontend-75669c8cd8-5kkmk

In [126]:
!kubectl get pods

NAME                        READY   STATUS    RESTARTS   AGE
fastapi-677965765c-z5kc6    1/1     Running   0          77m
frontend-74b7bcf65b-lknld   1/1     Running   0          33m


In [60]:
!kubectl get pvc

NAME      STATUS   VOLUME   CAPACITY   ACCESS MODES   STORAGECLASS   VOLUMEATTRIBUTESCLASS   AGE
pvc-llm   Bound    pv-llm   500Gi      RWO            manual         <unset>                 98m


In [61]:
!kubectl get pv

NAME     CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM             STORAGECLASS   VOLUMEATTRIBUTESCLASS   REASON   AGE
pv-llm   500Gi      RWO            Retain           Bound    default/pvc-llm   manual         <unset>                          98m


In [95]:
!kubectl logs $pod_name


== CUDA ==

CUDA Version 12.6.1

Container image Copyright (c) 2016-2023, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

This container image and its contents are governed by the NVIDIA Deep Learning Container License.
By pulling and using the container, you accept the terms and conditions of this license:
https://developer.nvidia.com/ngc/nvidia-deep-learning-container-license

A copy of this license is made available in this container at /NGC-DL-CONTAINER-LICENSE for your convenience.

2025-02-22 19:57:40.261589: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 19:57:40.277019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been regist

In [64]:
!kubectl describe pod $pod_name

Name:             fastapi-698c4dc749-l5qsk
Namespace:        default
Priority:         0
Service Account:  default
Node:             ip-10-0-1-228.ca-central-1.compute.internal/10.0.1.228
Start Time:       Sun, 09 Feb 2025 15:09:55 +0000
Labels:           app=fastapi
                  pod-template-hash=698c4dc749
Annotations:      <none>
Status:           Running
IP:               10.0.1.238
IPs:
  IP:           10.0.1.238
Controlled By:  ReplicaSet/fastapi-698c4dc749
Containers:
  fastapi:
    Container ID:   containerd://a15615861206a39f38823254d93970ca29766b92277c793c2856fb57bc0cfd11
    Image:          275678099358.dkr.ecr.ca-central-1.amazonaws.com/multi-agent/eberron-agent-server:25.02
    Image ID:       275678099358.dkr.ecr.ca-central-1.amazonaws.com/multi-agent/eberron-agent-server@sha256:0a7ec392240e2f7a4ba9ce05b279df68209e085074bbe4fb8f465e047156b2ff
    Port:           80/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Sun, 09 Feb 2025 15:14:49

In [ ]:
attachdetach-controller  AttachVolume.Attach failed for volume "pv-llm" : rpc error: code = Internal desc = Could not attach volume "volume_id" to node "i-055aa2853ecdeac3b": could not attach volume "volume_id" to node "i-055aa2853ecdeac3b": operation error EC2: AttachVolume, https response error StatusCode: 400, RequestID: a7250de3-c285-4963-84eb-5c46b28c2b96, api error InvalidParameterValue: The volume ID 'volume_id' is malformed

In [ ]:
!kubectl exec $pod_name bash -- nvidia-smi --version

In [63]:
!kubectl exec $pod_name bash -- nvidia-smi

Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
Mon Jan 13 22:15:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:1E.0 Off |                    0 |
| N/A   25C    P8             11W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |              

In [62]:
!kubectl exec $pod_name bash -- nvidia-smi -L

Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
GPU 0: Tesla T4 (UUID: GPU-fe1acad6-41f9-c441-44ed-d4ead6db6dc2)


In [61]:
!kubectl exec $pod_name bash -- nvcc --version

Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Aug_14_10:10:22_PDT_2024
Cuda compilation tools, release 12.6, V12.6.68
Build cuda_12.6.r12.6/compiler.34714021_0


In [ ]:
!kubectl describe pod $pod_name

In [113]:
!kubectl get pod -n kube-system

NAME                                   READY   STATUS    RESTARTS   AGE
aws-node-cvzp2                         2/2     Running   0          4m59s
aws-node-kmnjj                         2/2     Running   0          4m57s
aws-node-zlmn8                         2/2     Running   0          6m35s
coredns-749d5dbdd9-cdd8v               1/1     Running   0          8m29s
coredns-749d5dbdd9-xtrc8               1/1     Running   0          8m29s
ebs-csi-controller-59b6797bf-fhcph     5/5     Running   0          4m6s
ebs-csi-controller-59b6797bf-w9mns     5/5     Running   0          4m6s
ebs-csi-node-mp9sr                     3/3     Running   0          4m6s
ebs-csi-node-mqt55                     3/3     Running   0          4m6s
ebs-csi-node-q8dfl                     3/3     Running   0          4m6s
kube-proxy-ctjcd                       1/1     Running   0          6m35s
kube-proxy-nv8tx                       1/1     Running   0          4m57s
kube-proxy-qmwmh                       1/1   

In [177]:
!kubectl describe pvc pvc-llm

Name:          pvc-llm
Namespace:     default
StorageClass:  manual
Status:        Bound
Volume:        pv-llm
Labels:        <none>
Annotations:   pv.kubernetes.io/bind-completed: yes
               pv.kubernetes.io/bound-by-controller: yes
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      10Gi
Access Modes:  RWO
VolumeMode:    Filesystem
Used By:       kubyterlab-llm-pod-67f5cf95dc-s2mrn
Events:        <none>


In [178]:
!kubectl describe pv pv-llm

Name:            pv-llm
Labels:          <none>
Annotations:     pv.kubernetes.io/bound-by-controller: yes
Finalizers:      [kubernetes.io/pv-protection]
StorageClass:    manual
Status:          Bound
Claim:           default/pvc-llm
Reclaim Policy:  Retain
Access Modes:    RWO
VolumeMode:      Filesystem
Capacity:        10Gi
Node Affinity:   <none>
Message:         
Source:
    Type:       AWSElasticBlockStore (a Persistent Disk resource in AWS)
    VolumeID:   vol-04ce08a2c05b8e1da
    FSType:     ext4
    Partition:  0
    ReadOnly:   false
Events:         <none>


In [169]:
!kubectl describe deployment kubyterlab-llm-pod

Name:                   kubyterlab-llm-pod
Namespace:              default
CreationTimestamp:      Mon, 02 Dec 2024 20:05:56 +0000
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=kubyterlab-llm
Replicas:               1 desired | 1 updated | 1 total | 0 available | 1 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=kubyterlab-llm
  Containers:
   kubyterlab-llm:
    Image:      kubyterlab:24.10
    Port:       8888/TCP
    Host Port:  0/TCP
    Limits:
      cpu:             1
      memory:          8Gi
      nvidia.com/gpu:  1
    Requests:
      cpu:             1
      memory:          8Gi
      nvidia.com/gpu:  1
    Environment:
      JUPYTERLAB_SETTINGS_DIR:  /jupyterlab/config
      MISTRAL_MODEL:            /models/mistral
    Mounts:
      /corpus from pv-llm (rw)
      /jupyterlab from pv-llm

In [59]:
!kubectl get nodes

NAME                                          STATUS   ROLES    AGE   VERSION
ip-10-0-1-100.ca-central-1.compute.internal   Ready    <none>   47m   v1.30.7-eks-59bf375
ip-10-0-1-25.ca-central-1.compute.internal    Ready    <none>   47m   v1.30.7-eks-59bf375
ip-10-0-1-50.ca-central-1.compute.internal    Ready    <none>   48m   v1.30.7-eks-59bf375


In [58]:
!kubectl get configmap

NAME               DATA   AGE
kube-root-ca.crt   1      50m


In [57]:
!kubectl get all -l app.kubernetes.io/name=aws-ebs-csi-driver -n kube-system

NAME                                     READY   STATUS    RESTARTS   AGE
pod/ebs-csi-controller-59b6797bf-vfvs2   5/5     Running   0          47m
pod/ebs-csi-controller-59b6797bf-xt7fv   5/5     Running   0          47m
pod/ebs-csi-node-dwzz7                   3/3     Running   0          47m
pod/ebs-csi-node-gplqp                   3/3     Running   0          47m
pod/ebs-csi-node-l5nh6                   3/3     Running   0          47m

NAME                          DESIRED   CURRENT   READY   UP-TO-DATE   AVAILABLE   NODE SELECTOR            AGE
daemonset.apps/ebs-csi-node   3         3         3       3            3           kubernetes.io/os=linux   47m

NAME                                 READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/ebs-csi-controller   2/2     2            2           47m

NAME                                           DESIRED   CURRENT   READY   AGE
replicaset.apps/ebs-csi-controller-59b6797bf   2         2         2       47m


In [56]:
!aws iam list-open-id-connect-providers

{
    "OpenIDConnectProviderList": [
        {
            "Arn": "arn:aws:iam::275678099358:oidc-provider/oidc.eks.ca-central-1.amazonaws.com/id/8D7619520212428BD59C46B20BF19338"
        }
    ]
}


In [51]:
!kubectl logs deployment/ebs-csi-controller -n kube-system -c ebs-plugin

Found 2 pods, using pod/ebs-csi-controller-59b6797bf-vfvs2
I0113 21:28:06.936951       1 main.go:153] "Initializing metadata"
I0113 21:28:06.943481       1 metadata.go:48] "Retrieved metadata from IMDS"
I0113 21:28:06.944585       1 driver.go:69] "Driver Information" Driver="ebs.csi.aws.com" Version="v1.38.1"
I0113 22:01:16.236299       1 controller.go:410] "ControllerPublishVolume: attaching" volumeID="vol-0ed5d3cc8cb5a989e" nodeID="i-0fd8ede93d6ac59a4"
I0113 22:01:17.882498       1 controller.go:419] "ControllerPublishVolume: attached" volumeID="vol-0ed5d3cc8cb5a989e" nodeID="i-0fd8ede93d6ac59a4" devicePath="/dev/xvdaa"


In [52]:
!kubectl logs daemonset/ebs-csi-node -n kube-system -c ebs-plugin

Found 3 pods, using pod/ebs-csi-node-l5nh6
I0113 21:27:55.828513       1 main.go:153] "Initializing metadata"
I0113 21:27:55.832956       1 metadata.go:48] "Retrieved metadata from IMDS"
I0113 21:27:55.833450       1 driver.go:69] "Driver Information" Driver="ebs.csi.aws.com" Version="v1.38.1"
E0113 21:27:56.896438       1 node.go:856] "Unexpected failure when attempting to remove node taint(s)" err="isAllocatableSet: driver not found on node ip-10-0-1-50.ca-central-1.compute.internal"
I0113 21:27:57.410789       1 node.go:936] "CSINode Allocatable value is set" nodeName="ip-10-0-1-50.ca-central-1.compute.internal" count=24
I0113 22:01:18.817732       1 node.go:204] "NodeStageVolume: invalid partition config, will ignore." partition="0"
I0113 22:01:18.980008       1 mount_linux.go:295] Detected OS without systemd


In [53]:
!kubectl logs deployment/ebs-csi-controller -n kube-system -c csi-provisioner

Found 2 pods, using pod/ebs-csi-controller-59b6797bf-vfvs2
W0113 21:28:09.566495       1 feature_gate.go:354] Setting GA feature gate Topology=true. It will be removed in a future release.
I0113 21:28:09.566789       1 feature_gate.go:387] feature gates: {map[Topology:true]}
I0113 21:28:09.566840       1 csi-provisioner.go:154] Version: v5.1.0
I0113 21:28:09.566851       1 csi-provisioner.go:177] Building kube configs for running in cluster...
I0113 21:28:09.568886       1 common.go:143] "Probing CSI driver for readiness"
I0113 21:28:09.598446       1 csi-provisioner.go:230] Detected CSI driver ebs.csi.aws.com
I0113 21:28:09.598619       1 csi-provisioner.go:240] Supports migration from in-tree plugin: kubernetes.io/aws-ebs
I0113 21:28:09.600672       1 common.go:143] "Probing CSI driver for readiness"
I0113 21:28:09.603385       1 csi-provisioner.go:299] CSI driver supports PUBLISH_UNPUBLISH_VOLUME, watching VolumeAttachments
I0113 21:28:09.604750       1 controller.go:744] "Using sav

In [54]:
!kubectl logs deployment/ebs-csi-controller -n kube-system -c csi-attacher

Found 2 pods, using pod/ebs-csi-controller-59b6797bf-vfvs2
I0113 21:28:12.211572       1 main.go:109] "Version" version="v4.7.0"
I0113 21:28:12.215865       1 common.go:143] "Probing CSI driver for readiness"
I0113 21:28:12.238251       1 main.go:169] "CSI driver name" driver="ebs.csi.aws.com"
I0113 21:28:12.240448       1 common.go:143] "Probing CSI driver for readiness"
I0113 21:28:12.242734       1 main.go:249] "CSI driver supports ControllerPublishUnpublish, using real CSI handler" driver="ebs.csi.aws.com"
I0113 21:28:12.243624       1 leaderelection.go:254] attempting to acquire leader lease kube-system/external-attacher-leader-ebs-csi-aws-com...
I0113 21:28:12.265555       1 leaderelection.go:268] successfully acquired lease kube-system/external-attacher-leader-ebs-csi-aws-com
I0113 21:28:12.266787       1 leader_election.go:184] "became leader, starting"
I0113 21:28:12.266811       1 controller.go:129] "Starting CSI attacher"
I0113 21:28:12.267257       1 envvar.go:172] "Feature

In [55]:
!kubectl get sa ebs-csi-controller-sa -n kube-system -o jsonpath='{.metadata.annotations}'

{"eks.amazonaws.com/role-arn":"","meta.helm.sh/release-name":"aws-ebs-csi-driver","meta.helm.sh/release-namespace":"kube-system"}